# Linear Regression with ZKML


## Create and export model via Torch -> ONNX -> TFLite

In [ ]:
# check if notebook is in colab
try:
    import google.colab
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "onnx"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "hummingbird-ml"])


# rely on local installation of ezkl if the notebook is not in colab
except:
    pass

import tensorflow as tf
import torch
import onnx
import os
import sys
from onnx_tf.backend import prepare
from hummingbird.ml import convert
import numpy as np
from sklearn.linear_model import LinearRegression
sys.path.insert(0, "../..")
from zkml.python.converter import Converter
import pathlib

TFLITE_PATH = os.path.join(os.getcwd(), "linear_regression_model.tflite")
OUT_MODEL_PATH= os.path.join(os.getcwd(), "linear_regression_model.msgpack")
OUT_CONFIG_PATH= os.path.join(os.getcwd(), "linear_regression_model.json")
OUT_INPUT_NPY_PATH= os.path.join(os.getcwd(), "linear_regression_model_input.npy")
OUT_INPUT_MSG_PATH = os.path.join(os.getcwd(), "linear_regression_model_input.msgpack")
INPUT_CONVERTER_PATH = os.path.join(os.getcwd(), "../../src/zkml/python/input_converter.py")
TIME_CIRCUIT_PATH= os.path.join(os.getcwd(), "../../src/zkml/target/release/time_circuit")

x = [-1, 0, 1, 2, 3, 4]
y = [-3, -1, 1, 3, 5, 7]


model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(1,)),
    tf.keras.layers.Dense(1)
])
model.compile(optimizer="sgd", loss="mean_squared_error")
model.fit(x, y, epochs=200, verbose=1)
tf.saved_model.save(model, "linear_regression_tf")
converter = tf.lite.TFLiteConverter.from_saved_model("linear_regression_tf")
tflite_model = converter.convert()
tflite_model_file = pathlib.Path(TFLITE_PATH)
tflite_model_file.write_bytes(tflite_model)


## Convert model to msgpack using ZKML

In [ ]:
converter = Converter(model_path=TFLITE_PATH, expose_output=False, commit=False, scale_factor=7, k=6, num_cols=2, num_randoms=64, use_selectors=True)

model_packed, config_packed = converter.to_msgpack(
  start_layer=0,
  end_layer=10000,
)
if model_packed is None:
  raise Exception('Failed to convert model')
with open(OUT_MODEL_PATH, 'wb') as f:
  f.write(model_packed)
with open(OUT_CONFIG_PATH, 'wb') as f:
  f.write(config_packed)

## Create msgpack from input

In [ ]:
with open (OUT_INPUT_NPY_PATH, 'wb') as f:
    np.save(f, np.array(x[0]).reshape(-1, 1))

!python {INPUT_CONVERTER_PATH} --model_config {OUT_MODEL_PATH} --inputs {OUT_INPUT_NPY_PATH} --output {OUT_INPUT_MSG_PATH}


## Generate the proof

In [ ]:
#!../../src/zkml/target/release/time_circuit {OUT_MODEL_PATH} {OUT_INPUT_MSG_PATH} kzg
!../../src/zkml/target/release/test_circuit {OUT_MODEL_PATH} {OUT_INPUT_MSG_PATH} kzg
